### Cargar los datos del contenedor bronze

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [0]:
genre_schema = StructType([
    StructField("genreId", IntegerType(), True),
    StructField("genreName", StringType(), True)
])

In [0]:
genre_dt = spark.read.csv(f"{bronze_folder_path}/{v_file_date}/genre.csv", header=True, schema=genre_schema)

In [0]:
genre_dt_renamed = genre_dt.withColumnsRenamed({"genreId": "genre_id", "genreName": "genre_name"})

In [0]:
from pyspark.sql.functions import lit, current_timestamp

genre_dt_final = add_ingestion_date(genre_dt_renamed).withColumns({ "environment": lit(v_environment), "file_date": lit(v_file_date)})

In [0]:
display(genre_dt_final)

In [0]:
genre_dt_final.write.mode("overwrite").format("delta").saveAsTable("movie_silver.genre")

In [0]:
%sql
SELECT * FROM movie_silver.genre

In [0]:
genre_parquet = spark.read.format("delta").load("/mnt/moviehistoryrodrigopenna/silver/genre")
display(genre_parquet)